____

__Universidad Tecnológica Nacional, Buenos Aires__<br/>
__Ingeniería Industrial__<br/>
__Cátedra de Ciencia de Datos - Curso I5521 - Turno sabado mañana__<br/>
__clase_08: Practica Clustering: Wine Dataset__<br/>
__Elaborado por: Nicolas Aguirre__<br/>
____

# Importamos Librerias

In [ ]:
# Importamos las librerías necesarias para trabajar.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
# Importamos librerias de Clustering
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score, rand_score
# Importamos librerias de PCA
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Importamos Dataset

**Repositorio del Dataset**

https://archive.ics.uci.edu/ml/datasets/Wine

**Wine Data Set:**

https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data


In [ ]:
# Nombres de columnas
names_col = ['G','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols',
             'Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue',
             'OD280/OD315 of diluted wines','Proline']

wine_df = pd.read_csv('wine.data', delimiter=',', names=names_col)

In [ ]:
wine_df.head()

In [ ]:
wine_df.shape

In [ ]:
wine_df.isnull().sum()

In [ ]:
# Definimos las features que corresponden a X
x = wine_df.iloc[:,1:]
y = wine_df.iloc[:,0]
display(x.head())
print(x.shape)

In [ ]:
# Vamos a crear un dataframe para guardar los resultados
results_df = pd.DataFrame(columns=['Cluster','Rand_','Sil_'])

# Auto-Scaling

In [ ]:
# Realizamos un autoscaling con los datos, para todas las features
scaler = StandardScaler().fit(x)
xscal = scaler.transform(x)

# K-Means

In [ ]:
dist_cent = []
sil_list = []
rand_list = []
for k in range(2, 10):
    # Creamos el objecto de cluster y lo fiteamos en la misma linea utilizado xscal
    kmeans = KMeans(n_clusters=k, random_state=1).fit(xscal)
    centers_i = kmeans.cluster_centers_ # Centroide de cada cluster
    labels_i = kmeans.labels_ # Labels de cada muestra    
    # Silhouttte Score
    sil_score_i = silhouette_score(xscal,labels_i)
    sil_list.append(sil_score_i)        
    # Rand_Index
    rand_index_i = rand_score(y,labels_i)
    rand_list.append(rand_index_i)
    dist_cent.append(kmeans.inertia_)    
# Plot de metricas
fig, axs = plt.subplots(1,3,figsize=(15,5))    
axs[0].plot(range(2, 10), dist_cent, marker='s');
axs[0].set_xlabel('N° K')
axs[0].set_ylabel('Sum of squared distances')
# Silhoute plot
axs[1].plot(range(2, 10), sil_list, marker='s');
axs[1].set_xlabel('N° K')
axs[1].set_ylabel('Silhouette')
# Rand Index plot
axs[2].plot(range(2, 10), rand_list, marker='s');
axs[2].set_xlabel('N° K')
axs[2].set_ylabel('Rand Index')    
plt.show()


## P.C.A.


**Generamos un PCA con los datos luego del autoscaling**

In [ ]:
# Definimos la cantidad de componentes
n_comps = 13
components = range(1,n_comps + 1)
#Creamos el objeto PCA
pca = PCA(n_components=n_comps)

# Ajustamos
pca.fit(xscal)
# Transformamos
xpca = pca.transform(xscal)

# Porcentaje de la varianza explicada por cada Principal Component (PC)
eigenvalues = pca.explained_variance_ratio_

# Suma acumulada
eigenvalues_acum = pca.explained_variance_ratio_.cumsum() 

# Graficamos
# Eje Izquierdo
fig, ax1 = plt.subplots(figsize=(12,6))
ax1.set_xlabel('Componentes Principales',fontsize=20)
ax1.set_ylabel('Varianza Explicada', color='k',fontsize=20)
ax1.bar(components, eigenvalues, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Eje derecho
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Varianza Acumulada', color='k',fontsize=20) 
ax2.plot(components, eigenvalues_acum, color='red') 
ax2.tick_params(axis='y', labelcolor='red')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

### Eigenvectors ($\mathbf v$)

In [ ]:
# De la libreria:
# 'Principal axes in feature space, representing the directions of maximum variance in the data'
# The components are sorted by explained_variance_
pd.DataFrame(pca.components_[0:n_comps,:],columns=x.columns)

# En criollo:
# Es la direccion de los ejes de cada componente (autovectores)

In [ ]:
# Scatter plot de los datos, solamente con 2 PC
plt.figure(figsize=(9,6))
plt.scatter(xpca[:,0],xpca[:,1])    
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Figura de PC1 y PC2')
plt.show()

**Pregunta: Que cantidad de clusters seleccionariamos? Porque?**

In [ ]:
# Generamos un modelo de K means con ##RESPONDER## clusters con los datos autoscalados
rta = ##RESPONDER##
kmeans = KMeans(n_clusters = rta,
                random_state = 10).fit(xscal)

In [ ]:
# Visualizamos los centroides finales de cada cluster
centers = kmeans.cluster_centers_
centers

In [ ]:
# Scatter plot de muestras y centroides con 2 PC: segun Cluster verdadero vs Clustering con K-Means

#Verdadero
plt.figure(figsize=(9,6))
plt.scatter(xpca[:,0],xpca[:,1],c=wine_df['G'].astype(float))    
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Clustering Verdadero')
# K-Means
plt.figure(figsize=(9,6))
plt.scatter(xpca[:,0],xpca[:,1],c=kmeans.labels_.astype(float))
plt.scatter(centers[:,0], centers[:,1], marker="x", color='r',s=150)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Clustering K-Means')
plt.show()

**PREGUNTA:**

**¿Son correctos los centroides?**

## Métricas de K-means

In [ ]:
# Silhouttte Score
sil_score = silhouette_score(xscal,kmeans.labels_)
# Rand_Index
rand_index = rand_score(y,kmeans.labels_)

#Guardamos los resultados
results_df = results_df.append({'Cluster':'Kmeans',
                                'Rand_':rand_index,
                                'Sil_':sil_score},ignore_index=True)
results_df

# PCA + K-Means

**Ahora vamos dejar de usar las variables originales.**

**Vamos a clusterizar con lo que nos puedan explicar UNICAMENTE las primeras 2 PC, y compararemos los resultados.**

## 2 PC

In [ ]:
# Cantidad de PC quer queremos
reduced_dim = 2
# Definimos nuetro nuevo X de dimension reducida
xpca_rd = xpca[:,0:reduced_dim]

In [ ]:
# Generamos el modelo PCA + K-means
kmeans_rd = KMeans(n_clusters=3, 
                   random_state=10).fit(xpca_rd)

In [ ]:
# visualizamos los centroides finales de cada cluster
centers_rd = kmeans_rd.cluster_centers_
centers_rd

**Scatter plot con 2 PC:**

**Cluster verdadero vs Clustering con PCA+K-Means**

In [ ]:
#Verdadero
plt.figure(figsize=(9,6))
plt.scatter(xpca[:,0],xpca[:,1],c=wine_df['G'].astype(float))    
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Clustering Verdadero')
# PCA + K-Means
plt.figure(figsize=(9,6))
plt.scatter(xpca_rd[:,0],xpca_rd[:,1],c=kmeans_rd.labels_.astype(float))
plt.scatter(centers_rd[:,0], centers_rd[:,1], marker="x", color='r',s=150)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Clustering K-means+RD')
plt.show()

**PREGUNTA:**

**¿Y aca que sucedió?**

### Metricas

In [ ]:
# Silhouttte Score
sil_score = silhouette_score(xpca_rd,kmeans_rd.labels_)
# Rand_Index
rand_index = rand_score(y,kmeans_rd.labels_)

#Guardamos los resultados
results_df = results_df.append({'Cluster':'2 PC + Kmeans',
                                'Rand_':rand_index,
                                'Sil_':sil_score},ignore_index=True)
results_df

In [ ]:
pd.DataFrame(pca.components_[0:reduced_dim,:],columns=x.columns)

## 3 PC

In [ ]:
# Cantidad de PC quer queremos
reduced_dim = 3
xpca_rd = xpca[:,0:reduced_dim]
# Generamos el modelo PCA + K-means 
kmeans_rd = KMeans(n_clusters=3, 
                   random_state=10).fit(xpca_rd)
centers_rd = kmeans_rd.cluster_centers_
centers_rd

%matplotlib notebook
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(projection='3d')
ax.scatter3D(xpca[:,0], xpca[:,1],xpca[:,3], c=kmeans_rd.labels_)
ax.scatter3D(centers_rd[:,0], centers_rd[:,1],centers_rd[:,2], marker="o", color='r',s=150)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
plt.show()

### Metricas

In [ ]:
# Silhouttte Score
sil_score = silhouette_score(xpca_rd,kmeans_rd.labels_)
# Rand_Index
rand_index = rand_score(y,kmeans_rd.labels_)

#Guardamos los resultados
results_df = results_df.append({'Cluster':'3 PC + Kmeans',
                                'Rand_':rand_index,
                                'Sil_':sil_score},ignore_index=True)
results_df

**Pregunta:**

**Que Conclusiones sacamos ??**

**Cual es la cantidad maxima de PC que podriamos usar? Porque?**

# Clustering Jerárquico

In [ ]:
%matplotlib inline

# 3 PC
reduced_dim = 3
xpca_rd = xpca[:,0:reduced_dim]

# Definimos el linkage
Z = linkage(xpca_rd, method = 'ward',metric='euclidean')
# Threshold (Similaridad)
dist_cluster = 12
plt.figure(figsize=(25, 10))
dendrogram(Z,color_threshold=dist_cluster)
plt.axhline(c='k',linestyle='--', y=dist_cluster)
plt.show()

In [ ]:
%matplotlib inline

dist_cluster = 12
plt.figure(figsize=(25, 10))
dendrogram(Z,color_threshold=dist_cluster)
plt.axhline(c='k',linestyle='--', y=dist_cluster)
plt.show()

cluster = AgglomerativeClustering(n_clusters=None, distance_threshold =dist_cluster,
                                  affinity='euclidean',
                                  linkage='ward')
cluster.fit_predict(xpca_rd)

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(projection='3d')
ax.scatter3D(xpca[:,0], xpca[:,1],xpca[:,3], c=cluster.labels_)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
plt.show()

## Metricas

In [ ]:
sil_score = silhouette_score(xpca_rd,cluster.labels_)
# Rand_Index
rand_index = rand_score(y,cluster.labels_)

#Guardamos los resultados
results_df = results_df.append({'Cluster':str( reduced_dim) +  ' PC + Hierarchy ',
                                'Rand_':rand_index,
                                'Sil_':sil_score},ignore_index=True)


# Resultados

In [ ]:
results_df

# Preguntas ?

![meme](https://memegenerator.net/img/instances/72560208/one-does-not-simply-run-principal-component-analysis-on-a-data-set-with-106-rows-and-103-variables.jpg)

# Recontruccion

**Si quisieramos volver a nuestro espacio original, consideremos que:**

$Z = X \cdot \mathbf{V} \rightarrow \hat X = Z \cdot \mathbf{V}^T$

siendo $Z$ la proyeccion de X en el nuevo subspace.

Como vimos en la clase, los datos tienen que ser centrados (como minimo!)

Para cumplir con las ecuaciones y volver a tener nuestra $\hat X$, deberiamos sumarle las medias ...

**A)** $\hat X = Z \cdot \mathbf{V}^T + \mu$

Pero en nuestro caso Standarizamos los datos ($\mu = 0, \sigma = 1$). Entonces debemos re escalar teniendo en cuenta $\sigma$ antes de sumar $\mu$.

**B)** $\hat X = ( Z  \cdot \mathbf{V}^T) \cdot \sigma + \mu $

In [ ]:
mu = np.mean(x, axis=0)
std = np.std(x, axis=0)[:,np.newaxis].T

# 2 PCA
reduced_dim = 2
xpca_rd = xpca[:,0:reduced_dim]
x_rec = np.dot(xpca_rd, pca.components_[0:reduced_dim,:])
x_rec = x_rec * std
x_rec += mu

x_rec_df = pd.DataFrame(x_rec,columns=x.columns)

In [ ]:
# Samples de la reconstruccion
sample = np.random.randint(0,high=x.shape[0])

display(x.iloc[sample,:].to_frame().transpose())
display(x_rec_df.iloc[sample,:].to_frame().transpose())

**Resumen**:

* Cuando los datos son centrados, la reconstruccion se hace teniendo en cuenta que:

**A)**  $\hat X = Z \cdot \mathbf{V}^T + \mu $


* Cuando ademas los datos son standarizados, la reconstruccion se hace teniendo en cuenta que:

**B)**   $\hat X = (Z \cdot \mathbf{V}^T) \cdot \sigma  + \mu $

**Preguntas:**

**Cuando debemos centrar y cuando estandarizar?**

**La presencia de outliers puede modificar el resultado del PCA? Porque?**

**Podemos hacer PCA sobre features categoricas?**